# Anomaly Detection for 5G Performance Assurance

The accessibility of 5G technology is a crucial kpi that has a significant impact on both user experience and customer loyalty. To ensure the smooth functioning of the 5G system, various factors such as 5G radio parameters, random access control, paging control, and admission control must be taken into consideration. In this demo, we will use a 5G cell tower data to build a machine learning model predict anomalies in 5G accessibility. This will allow the operations team to proactively perform maintenance and prevent potential issues that may lead to customer dissatisfaction and service churn.

##### Data Scientist Experimentation
- [Data Preparation W/ Data Wrangler](#Preprocessing-&-feature-engineering)
- [Feature Management W/ Feature Store](#Pull-data-from-offline-feature-store)
- [Experiment & Deployment on SageMaker](#Train-XGBoost-Model)
   - [Xgboost Model](#Train-XGBoost-Model)
   - [Isolation Forest Model](#Train-Isolation-Forest-Model)
- [Model Monitoring](#Model-Monitoring)

**This Demo is optimized for SageMaker Studio using Studio notebook in Data Science Kernel**

### Setup

Install required and/or update libraries

In [2]:
import sys

!{sys.executable} -m  pip install -Uq pip --quiet

!{sys.executable} -m pip install -Uq awswrangler sagemaker boto3 --quiet

### Import & Global Parameters

In [3]:
import boto3
import sagemaker
import pandas as pd

sagemaker_session = sagemaker.Session()

region = sagemaker_session.boto_region_name
sagemaker_role = sagemaker.get_execution_role()

bucket = sagemaker_session.default_bucket()

s3_client = boto3.client("s3", region_name=region)
sagemaker_client = boto3.client("sagemaker")

prefix = "telco-anomaly-demo"

## Dataset

To determine good 5G accessibility, we will use `abnormal_release_rate`. Any abnoraml_release_rate  > 0 is considered high probability for anomaly.  Then we will train a classification model that can predict the likelihood of connectivity drops base on input features like `network utilization`, `contention rates`, `health index`, and `throughput` parameters. This use case is part of the 5G performance assurance initiative, aimed at predicting any potential loss of connectivity to the 5G radio network in the next hour, helping to ensure a seamless and uninterrupted user experience.

abnormal_release_rate (likelihood of connectivity drops)

under known conditions:

- network utilization
- contention rates, 
- health index,
- throughput parameters

In [4]:
df = pd.read_csv("data/tlc303-5gcell.csv")
df.head()

,cellname_nrcell,Location,cell_ID,date_extracted,hour_extracted,5g_ran_health_index_mwc_1,5g_availability_mwc_1,5g_accessibility_mwc_1,5g_mobility_mwc_1,5g_retainability_mwc_1,...,5g_initial_uplink_bler_num_huaw,5g_uplink_resource_block_utilization_den_huaw,5g_uplink_resource_block_utilization_num_huaw,5g_user_downlink_avg_throughput_den_huaw,5g_user_downlink_avg_throughput_num_huaw,5g_user_uplink_avg_throughput_den_huaw,5g_user_uplink_avg_throughput_num_huaw,sqa_5g_initial_downlink_bler,sqa_5g_initial_uplink_bler,_c54
0,BULUA2N-403_4RFS,BULUA2N,403_4RFS,05/11/2021,12,1.0,1.0,1.0,1.00,1.0,...,234.0,51.0,23.12,3517500.0,88243.0,3668000.0,7246.60,0.10,0.0,NaN
1,BULUACDON-401,BULUACDON,401,05/11/2021,12,1.0,1.0,1.0,1.00,1.0,...,2125.0,51.0,22.47,61353000.0,3022105.0,35390000.0,69801.45,0.11,0.0,NaN
2,BULUACDON-402,BULUACDON,402,05/11/2021,12,1.0,1.0,1.0,1.00,1.0,...,1881.0,51.0,23.29,86807000.0,3604944.0,76525500.0,245608.71,0.05,0.0,NaN
3,BULUACDON-403,BULUACDON,403,05/11/2021,12,1.0,1.0,1.0,0.99,1.0,...,621.0,51.0,21.22,3338000.0,138399.0,2956500.0,7258.59,0.05,0.0,NaN
4,BUNGADQCR-401_4RFS,BUNGADQCR,401_4RFS,05/11/2021,12,1.0,1.0,1.0,1.00,1.0,...,47.0,51.0,21.04,2274500.0,86763.0,1677000.0,6453.86,0.00,0.0,NaN


Upload the data to S3 bucket for SageMaker Data Wrangler

In [5]:
cell5g_s3_key = f"{prefix}/data/raw/tlc303-5gcell.csv"

s3_client.upload_file(
    Filename="data/tlc303-5gcell.csv", Bucket=bucket, Key=cell5g_s3_key
)

cell5g_s3_path = f's3://{bucket}/{cell5g_s3_key}'

print(f'Raw 5g dataset is located at {cell5g_s3_path}')

Raw 5g dataset is located at s3://sagemaker-us-west-2-987720697751/telco-anomaly-demo/data/raw/tlc303-5gcell.csv


### Preprocessing & feature engineering
From here let's jump into SageMaker Data Wranger (DW) to preprocess our dataset.  In this step, we are performing the following task. keep in mind your source data location may be different in your environment. You can edit the data source in the DW work environment.

1. Ingest data from S3
2. visualize and analyze our data
3. process and transform to clean up and encode our dataset
5. export data to feature store

<img src="statics/data_wrangler.png"  width="75%" height="75%">


While Wait for the Data To Load, Let's Explore the Feature Store Console
Feature Store

<img src="statics/feature_store.png"  width="75%" height="75%">

### Pull data from offline feature store
----
We can now build our training and test datasets repeatedly and consistenly from the feature groups we just created.  In this example, we will submit a SQL query to join the the Claims and Customers features.

In [6]:
from sagemaker.feature_store.feature_group import FeatureGroup

anomaly_features = FeatureGroup(name="5gcell-anomaly-features", sagemaker_session=sagemaker_session)

query = anomaly_features.athena_query()

table_name = query.table_name
                       
query_string = f"""
SELECT * FROM "{table_name}"
"""

query.run(query_string=query_string, output_location=f"s3://{bucket}/{prefix}/data/query_results")
query.wait()

dataset = query.as_dataframe()

dataset

,health,accessibility,5g_users,contention_rate,utilization,downlink_throughput,uplink_throughput,anomaly,location_id,eventtime,write_time,api_invocation_time,is_deleted
0,1.00,1.00,0.000320,0.000454,0.277624,0.000072,0.000191,0,BUDLAANCEBN_401,1.676148e+09,2023-02-11 20:44:39.332,2023-02-11 20:39:38.000,False
1,1.00,1.00,0.114633,0.014966,0.425414,0.030132,0.122794,0,BUHAN3N_403_4RFS,1.676148e+09,2023-02-11 20:44:39.332,2023-02-11 20:39:38.000,False
2,1.00,0.99,0.040026,0.000907,0.327348,0.007802,0.079614,0,BUTINGN_401_4RFS,1.676148e+09,2023-02-11 20:44:39.332,2023-02-11 20:39:38.000,False
3,1.00,1.00,0.001921,0.000000,0.142265,0.000996,0.000691,0,BASILIOSTMALNCRR-402_4RFS_None,1.676148e+09,2023-02-11 20:44:39.332,2023-02-11 20:39:38.000,False
4,1.00,1.00,0.000961,0.000000,0.138122,0.000004,0.000273,0,BASILIOSTMALNCRR-402_4RFS_None,1.676148e+09,2023-02-11 20:44:39.332,2023-02-11 20:39:38.000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
718674,1.00,1.00,0.088056,0.019501,0.360497,0.009696,0.057085,0,BALINGASAN_403_4RFS,1.676328e+09,2023-02-13 22:50:09.991,2023-02-13 22:46:31.000,False
718675,1.00,1.00,0.001921,0.000000,0.287293,0.000206,0.001956,0,BALIOKN_401,1.676328e+09,2023-02-13 22:50:09.991,2023-02-13 22:46:31.000,False
718676,0.96,0.98,0.021134,0.003855,0.292818,0.005578,0.010421,0,BALULAM_353,1.676328e+09,2023-02-13 22:50:09.991,2023-02-13 22:46:31.000,False
718677,1.00,1.00,0.085175,0.001361,0.306630,0.012122,0.049172,0,BALUTMLN_403_4RFS,1.676328e+09,2023-02-13 22:50:09.991,2023-02-13 22:46:31.000,False


### Train XGBoost Model
----
#### Move the target varibale to the first column for our xgboost model

Split train & test dataset

In [7]:
col_order = ["anomaly"] + list(dataset.drop(["location_id", "anomaly", "eventtime", "write_time","api_invocation_time",'is_deleted'], axis=1).columns)

train = dataset.sample(frac=0.80, random_state=0)[col_order]
test = dataset.drop(train.index)[col_order]

In [8]:
train.to_csv("data/train.csv", index=False)
key = f"{prefix}/data/xgboost/train.csv"

s3_client.upload_file(
    Filename="data/train.csv",
    Bucket=bucket,
    Key=key,
)

train_s3_path = f"s3://{bucket}/{key}"
print(f"training data is uploaded to {train_s3_path}")

training data is uploaded to s3://sagemaker-us-west-2-987720697751/telco-anomaly-demo/data/xgboost/train.csv


#### Set the hyperparameters
These are the parameters which will be sent to our training script in order to train the model. Although they are all defined as "hyperparameters" here, they can encompass XGBoost's [Learning Task Parameters](https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters), [Tree Booster Parameters](https://xgboost.readthedocs.io/en/latest/parameter.html#parameters-for-tree-booster), or any other parameters you'd like to configure for XGBoost.

#### Setup Experiment Run Context

In [27]:
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.experiments.run import Run, load_run
from sagemaker.utils import unique_name_from_base

train_instance_count=1
train_instance_type="ml.m5.xlarge" 

experiment_name = unique_name_from_base(prefix)

run_name = unique_name_from_base("xgboost-experiment")

with Run(experiment_name=experiment_name, run_name=run_name, 
         sagemaker_session=sagemaker_session) as run:
        
    run.log_file("data/train.csv", is_output=False)
    
    hyperparameters = {
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
        "region":region
    }

    xgb_estimator = XGBoost(
        entry_point="xgboost_starter_script.py",
        source_dir="code",
        hyperparameters=hyperparameters,
        role=sagemaker_role,
        instance_count=train_instance_count,
        instance_type=train_instance_type,        
        framework_version="1.5-1",
    )
    
    xgb_estimator.fit(inputs={"train": train_s3_path})

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.xlarge.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-02-14-16-42-09-588


2023-02-14 16:42:10 Starting - Starting the training job...
2023-02-14 16:42:27 Starting - Preparing the instances for training...
2023-02-14 16:43:08 Downloading - Downloading input data...
2023-02-14 16:43:38 Training - Downloading the training image...
2023-02-14 16:43:59 Training - Training image download completed. Training in progress..[2023-02-14 16:44:04.631 ip-10-0-245-87.us-west-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-02-14:16:44:04:INFO] Imported framework sagemaker_xgboost_container.training
[2023-02-14:16:44:04:INFO] No GPUs detected (normal if no gpus installed)
[2023-02-14:16:44:04:INFO] Invoking user training script.
[2023-02-14:16:44:04:INFO] Module xgboost_starter_script does not provide a setup.py. 
Generating setup.py
[2023-02-14:16:44:04:INFO] Generating setup.cfg
[2023-02-14:16:44:04:INFO] Generating MANIFEST.in
[2023-02-14:16:44:04:INFO] Installing module with the following command:
/miniconda3/bin/python3 -m pip install .

#### Deploy model to an endpoint
We are going to enable data capturing for model monitoring

In [28]:
from sagemaker.serializers import CSVSerializer
from sagemaker.model_monitor import DataCaptureConfig

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=f"s3://{bucket}/{prefix}/monitoring/datacapture"
)


predictor = xgb_estimator.deploy(
    initial_instance_count=1, instance_type="ml.m5.xlarge", serializer=CSVSerializer(), data_capture_config=data_capture_config
)

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-02-14-16-46-04-027
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-02-14-16-46-04-027
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-02-14-16-46-04-027


-----!

#### Test inference on endpoint
Capture the confussion matrix results in the experiemnt for historic reference.

In [29]:
import numpy as np
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions =[]
    for array in split_array:
        predictions = predictions + sum(predictor.predict(array), [])

    return [float(i) for i in predictions]

def calibrate(probabilities, cutoff=.2):
    predictions = []
    for p in probabilities:
        if p <= cutoff:
            predictions.append(0)
        else:
            predictions.append(1)
    return predictions

In [30]:
with load_run(experiment_name=experiment_name, run_name=run_name) as run:

    # run batch prediction
    probabilities = predict(test.to_numpy()[:, 1:])
    # run calibration and visualize the results
    predictions = np.asarray(calibrate(probabilities, 0.4))
    run.log_confusion_matrix(test["anomaly"], predictions, unique_name_from_base("Confusion-Matrix"))

print(f"Experiment Name: {experiment_name}\n")

print(f"Run Name: {run_name}\n")

pd.crosstab(
    index=test.iloc[:, 0],
    columns=predictions,
    rownames=["actual"],
    colnames=["predictions"],
)

INFO:sagemaker.experiments.run:The run (xgboost-experiment-1676392928-8eb5) under experiment (telco-anomaly-demo-1676392928-cc66) already exists. Loading it. Note: sagemaker.experiments.load_run is recommended to use when the desired run already exists.


Experiment Name: telco-anomaly-demo-1676392928-cc66

Run Name: xgboost-experiment-1676392928-8eb5



predictions,0,1
actual,,
0,115159,6839
1,7361,14377


### Train Isolation Forest Model
----
This is an unsupervised approach. we will use the full dataset and try to isolate anomaly base on their deviation for norm.

In [31]:
iso_input = dataset.drop(["location_id", "anomaly", "eventtime", 
                          "write_time","api_invocation_time",'is_deleted'], axis=1)
iso_input.to_csv("data/iso_input.csv", index=False)
key = f"{prefix}/data/isoforest/iso_input.csv"

s3_client.upload_file(
    Filename="data/iso_input.csv",
    Bucket=bucket,
    Key=key,
)

input_s3_path = f"s3://{bucket}/{key}"
print(f"training data is uploaded to {input_s3_path}")

training data is uploaded to s3://sagemaker-us-west-2-987720697751/telco-anomaly-demo/data/isoforest/iso_input.csv


In [32]:
from sagemaker.sklearn.estimator import SKLearn

run_name = unique_name_from_base("isoforest-experiment")

with Run(experiment_name=experiment_name, run_name=run_name, 
         sagemaker_session=sagemaker_session) as run:
    
    run.log_file("data/iso_input.csv", is_output=False)
    FRAMEWORK_VERSION = "1.0-1"

    sklearn = SKLearn(
        entry_point="isolation_forest_script.py",
        source_dir="code",
        framework_version="1.0-1",
        instance_count=train_instance_count,
        instance_type=train_instance_type,
        role=sagemaker_role,
        sagemaker_session=sagemaker_session,
        hyperparameters={"max_samples": 512,
                        "random_state": 42,
                        "region":region},
    )
    sklearn.fit({"train": input_s3_path})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2023-02-14-16-48-50-034


2023-02-14 16:48:50 Starting - Starting the training job...
2023-02-14 16:49:05 Starting - Preparing the instances for training...
2023-02-14 16:49:47 Downloading - Downloading input data...
2023-02-14 16:50:12 Training - Downloading the training image...
2023-02-14 16:50:28 Training - Training image download completed. Training in progress.2023-02-14 16:50:38,856 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-02-14 16:50:38,858 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-02-14 16:50:38,866 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-02-14 16:50:39,077 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 668.0/668.0 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with statu

#### Deploy IsoForest Model in one step

In [33]:
from sagemaker.deserializers import JSONDeserializer

isoforest_predictor = sklearn.deploy(
    initial_instance_count=1, instance_type="ml.m5.xlarge", serializer=CSVSerializer(), deserializer = JSONDeserializer()
)

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2023-02-14-16-52-08-004
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2023-02-14-16-52-08-004
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2023-02-14-16-52-08-004


-----!

#### Test inference on endpoint
Capture the confussion matrix results in the experiemnt for historic reference.

In [34]:
with load_run(experiment_name=experiment_name, run_name=run_name) as run:

    results = isoforest_predictor.predict(test.to_numpy()[:, 1:])
    
    # run fix -1 value to 0
    predictions = []
    for x in results:
        if x <= 0:
            predictions.append(0)
        else:
            predictions.append(x)
            
    predictions = np.asarray(predictions)
    run.log_confusion_matrix(test["anomaly"], predictions, unique_name_from_base("IsoForest-Confusion-Matrix"))

print(f"Experiment Name: {experiment_name}\n")

print(f"Run Name: {run_name}\n")

pd.crosstab(
    index=test.iloc[:, 0],
    columns=predictions,
    rownames=["actual"],
    colnames=["predictions"],
)

INFO:sagemaker.experiments.run:The run (isoforest-experiment-1676393329-6878) under experiment (telco-anomaly-demo-1676392928-cc66) already exists. Loading it. Note: sagemaker.experiments.load_run is recommended to use when the desired run already exists.


Experiment Name: telco-anomaly-demo-1676392928-cc66

Run Name: isoforest-experiment-1676393329-6878



predictions,0,1
actual,,
0,7645,114353
1,5699,16039


### Model Monitoring

<img src="statics/Model_monitoring.png"  width="50%" height="50%">

#### 1. Create a baselining job with training dataset
Now that you have the training data ready in Amazon S3, start a job to suggest constraints. DefaultModelMonitor.suggest_baseline(..) starts a ProcessingJob using an Amazon SageMaker provided Model Monitor container to generate the constraints.

In [35]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

# this is our training dataset
baseline_data_uri = train_s3_path
baseline_results_prefix = f"{prefix}/monitoring/baselining/results"
baseline_results_uri = f"s3://{bucket}/{baseline_results_prefix}"


my_default_monitor = DefaultModelMonitor(
    role=sagemaker_role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True,
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2023-02-14-16-54-46-103


..........................2023-02-14 16:58:52,688 - matplotlib.font_manager - INFO - Generating new fontManager, this may take some time...
2023-02-14 16:58:53.210032: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-14 16:58:53.210064: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-14 16:58:54.740911: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-14 16:58:54.740939: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-14 16:58:54.740965: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be runn

In [36]:
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get("Key") for report_file in result.get("Contents")]
print("Found Files:")
print("\n ".join(report_files))

Found Files:
telco-anomaly-demo/monitoring/baselining/results/constraints.json
 telco-anomaly-demo/monitoring/baselining/results/statistics.json


In [37]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data,string_statistics.common.num_present,string_statistics.common.num_missing,string_statistics.distinct_count
0,anomaly,Integral,574943.0,0.0,0.151417,87056.000000,0.358455,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...",NaN,NaN,NaN
1,health,Fractional,574943.0,0.0,0.962546,553408.800000,0.143614,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[1.0, 1.0, 1.0, 1.0, 0.94, 1.0, 1.0, 0.98, 0....",NaN,NaN,NaN
2,accessibility,Fractional,574943.0,0.0,0.913594,525264.590000,0.263551,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[1.0, 1.0, 1.0, 1.0, 0.89, 0.99, 1.0, 0.97, 0...",NaN,NaN,NaN
3,5g_users,Fractional,574943.0,0.0,0.035167,20218.873519,0.055565,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0083253282100544, 0.0137688120397054, 0.04...",NaN,NaN,NaN
4,contention_rate,Fractional,574943.0,0.0,0.003228,1856.168481,0.012486,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.00045351473922902497, 0.00521541...",NaN,NaN,NaN
5,utilization,Fractional,574943.0,0.0,0.261142,150141.774862,0.100275,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.2845303867403315], [0.6436464088397791], [...",NaN,NaN,NaN
6,downlink_throughput,String,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,574943.0,0.0,93482.0
7,uplink_throughput,String,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,574943.0,0.0,95368.0


In [38]:
constraints_df = pd.io.json.json_normalize(
    baseline_job.suggested_constraints().body_dict["features"]
)
constraints_df.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,inferred_type,completeness,num_constraints.is_non_negative
0,anomaly,Integral,1.0,True
1,health,Fractional,1.0,True
2,accessibility,Fractional,1.0,True
3,5g_users,Fractional,1.0,True
4,contention_rate,Fractional,1.0,True
5,utilization,Fractional,1.0,True
6,downlink_throughput,String,1.0,NaN
7,uplink_throughput,String,1.0,NaN


#### 2. Create a schedule to analyze collected data for data quality issues

In [39]:
from time import gmtime, strftime
from sagemaker.model_monitor import CronExpressionGenerator

mon_schedule_name = unique_name_from_base(f"{prefix}-monitoring-job")

s3_report_path = f"s3://{bucket}/{prefix}/montoring/report"

my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint, #predictor endpoint name
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(), #Hourly
    enable_cloudwatch_metrics=True,
)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: telco-anomaly-demo-monitoring-job-1676394067-0efa


In [40]:
desc_schedule_result = my_default_monitor.describe_schedule()
print("Schedule status: {}".format(desc_schedule_result["MonitoringScheduleStatus"]))

Schedule status: Pending


### Generate some artificial traffic 

In [41]:
count = 0
for i in range(100):

    predict(test.to_numpy()[:, 1:])
    count+=1
    if count%10 == 0:
        print(f"predicting artificial traffic batch {count} ...")

predicting artificial traffic batch 10 ...
predicting artificial traffic batch 20 ...
predicting artificial traffic batch 30 ...
predicting artificial traffic batch 40 ...
predicting artificial traffic batch 50 ...
predicting artificial traffic batch 60 ...
predicting artificial traffic batch 70 ...
predicting artificial traffic batch 80 ...
predicting artificial traffic batch 90 ...
predicting artificial traffic batch 100 ...


In [42]:
mon_executions = my_default_monitor.list_executions()
mon_executions

## Clean Up

In [ ]:
feature_group_name = '<FEATURE GROUP NAME>'
sagemaker_client.delete_feature_group(
    FeatureGroupName= feature_group_name
)

In [71]:
import time
def remove_experiment(experiment_name):
    trials = sagemaker_client.list_trials(ExperimentName=experiment_name)['TrialSummaries']
    print('TrialNames:')
    for trial in trials:
        trial_name = trial['TrialName']
        print(f"\n{trial_name}")

        components_in_trial = sagemaker_client.list_trial_components(TrialName=trial_name)
        print('\tTrialComponentNames:')
        for component in components_in_trial['TrialComponentSummaries']:
            component_name = component['TrialComponentName']
            print(f"\t{component_name}")
            sagemaker_client.disassociate_trial_component(TrialComponentName=component_name, TrialName=trial_name)
            try:
                # comment out to keep trial components
                sagemaker_client.delete_trial_component(TrialComponentName=component_name)
            except:
                # component is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        sagemaker_client.delete_trial(TrialName=trial_name)
    sagemaker_client.delete_experiment(ExperimentName=experiment_name)
    print(f"\nExperiment {experiment_name} deleted")

experiment_name = 'telco-anomaly-demo-1675992634-110c'
remove_experiment(experiment_name)

TrialNames:

Default-Run-Group-telco-anomaly-demo-1675992634-110c
	TrialComponentNames:
	sagemaker-scikit-learn-2023-02-10-01-37-21-249-aws-training-job
	telco-anomaly-demo-1675992634-110c-isoforest-experiment-1675993040-4a95
	sagemaker-xgboost-2023-02-10-01-30-35-135-aws-training-job
	telco-anomaly-demo-1675992634-110c-xgboost-experiment-1675992634-e763

Experiment telco-anomaly-demo-1675992634-110c deleted


In [ ]:
def remove_endpoint(endpoint_name):
    monitor_schedules = sagemaker_client.list_monitoring_schedules(EndpointName=endpoint_name)['MonitoringScheduleSummaries']
    print('Monitoring Schedule:')
    for ms in monitor_schedules:
        ms_name = ms['MonitoringScheduleName']
        print(f"\n{ms_name}")

        sagemaker_client.delete_monitoring_schedule(MonitoringScheduleName=ms_name)
        
    sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
    print(f"Endpoint {endpoint_name} deleted")

#xgboost
remove_endpoint(predictor.endpoint_name)
# #isolation forest
remove_endpoint(isoforest_predictor.endpoint_name)